In [1]:
import pynbody
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
galaxy_masses=pd.DataFrame()
BH_halos=pd.DataFrame()
snap_id=0

In [3]:
iords=pd.read_csv('iords.csv')
masses=pd.read_csv('ALL_galaxy_masses.csv')

In [4]:
def loader(path):
    '''returns snapshot and halo'''
    snap=pynbody.load(path)
    snap.physical_units()
    h=snap.halos()
    return snap,h

def BH_finder(snap):
    '''returns subsnap of black holes'''
    return snap.star[pynbody.filt.BandPass('tform','-15 Gyr','0 Gyr')]

def analysis(halo, view):
    '''center snap on halo's center of mass (angmom gives errors if there aren't enough pixels)'''
    if view==1:
        pynbody.analysis.angmom.faceon(halo)
    elif view==2:
        pynbody.analysis.angmom.sideon(halo)
    else:
        pynbody.analysis.halo.center(halo)
        
def plot_BH_pos(h, BH, grpid, attribute, snapshot, cmap='Greys', w=100, save=False, view=0):
    '''plots position of black holes. Send halo array(h), black hole array(BH), grpid, attribute, and snapshot you want to plot 
    Optional: width/preferred scope(w) (default=100), colormap(default='Greys'), save(default=False)'''
    
    pynbody.plot.image(h[grpid].s,qty=attribute,cmap=cmap, width=w, log=False)
    plt.plot(np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[0],np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[1],'r+')
    plt.xlim(-w/2,w/2)
    plt.ylim(-w/2,w/2)
    plt.title('%s_h%s_%s_w%s'%(snapshot, grpid, attribute, w))
    
    if save==True:
        plt.savefig('plots/%s/h%s/h148_h%s_%s_w%s.png'%(snapshot, grpid, grpid, attribute, w))
    plt.show()
    
def add_mass_columns(hgrpid, grpid, snap_id):
    '''returns row with gas, stellar, and total masses for one halo tracked through time'''
    df=pd.DataFrame([[hgrpid.g['mass'].sum(), hgrpid.s['mass'].sum(), hgrpid['mass'].sum()]], index=[snap_id], columns=['H[%d] Gas'%(grpid),'H[%d] Star'%(grpid),'H[%d] Total'%(grpid)])
    df.units='Msol'
    return df

def mass_df(h, BHgrp, snap_id):
    df=pd.DataFrame()
    for i in BHgrp:
        df=pd.concat([df, add_mass_columns(h[i], i, snap_id)], axis=1, join='outer')
    
    df.units='Msol'
    return df

def galaxy_df(snap_id_array,h,BHgrp):
    galaxy_masses=pd.DataFrame()
    for i in snap_id_array:
        vars()['mass_%s'%(i)]=mass_df(h,BHgrp,i)
        galaxy_masses=galaxy_masses.append(vars()['mass_%s'%(i)])

    return galaxy_masses

def plot_SFH(h, grpid,  snapshot, ls='dashed', c='k', save=True, mf=False):
    '''plots SFH. Send halo array(h[grpid]), linestyle(ls=dashed), color(c=k) grpid, and snapshot you want to plot 
    Optional:save(default=True)'''
    plt.clf()
    pynbody.plot.sfh(h[grpid].s,linestyle=ls,color=c, massform=mf);
    plt.title('%s_h[%s]_SFH'%(snapshot,grpid), y=1.14);
    plt.savefig('plots/SFH/%s_h[%s]_SFH.png'%(snapshot, grpid));
    

In [5]:
#array of last four digits for each snap
snap_id_array=['0139','0225','0275','0640','0974','1024','1269','1280','1408','1740','2048','2088','2432','2688','2816','2944','3072','3195','3200','3328','3456','3584','3606','3712','3840','3968','4096']

In [6]:
#snap id index (negative values start from end)
snap_id=snap_id-1
snap_id_array[snap_id]

'4096'

In [7]:
#set path
path_lower_z='/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.00%s/h148.cosmo50PLK.3072g3HbwK1BH.00%s'%(snap_id_array[snap_id],snap_id_array[snap_id])
#set path
path_higher_z='/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.00%s/h148.cosmo50PLK.3072g3HbwK1BH.00%s'%(snap_id_array[snap_id-1],snap_id_array[snap_id-1])

In [8]:
snap_lower_z=pynbody.load(path_lower_z)
#snap_higher_z=pynbody.load(path_higher_z)

In [9]:
h= snap_lower_z.halos()

In [10]:
#returns subsnap of black holes
BH_lower_z=BH_finder(snap_lower_z)
BH_lower_z['amiga.grp']

SimArray([  1,   3,   1,   1, 649,   6,  11,  12,   1,   1,   1,   5])

In [11]:
masses=masses.rename(columns={'Unnamed: 0':'snap'})
masses=masses.set_index('snap')
masses

,H[109] Gas,H[109] Star,H[109] Total,H[10] Gas,H[10] Star,H[10] Total,H[115] Gas,H[115] Star,H[115] Total,H[11] Gas,H[11] Star,H[11] Total,H[12] Gas,H[12] Star,H[12] Total,H[130] Gas,H[130] Star,H[130] Total,H[131] Gas,H[131] Star,H[131] Total,H[13] Gas,H[13] Star,H[13] Total,H[141] Gas,H[141] Star,H[141] Total,H[145] Gas,H[145] Star,H[145] Total,H[149] Gas,H[149] Star,H[149] Total,H[14] Gas,H[14] Star,H[14] Total,H[157] Gas,H[157] Star,H[157] Total,H[158] Gas,H[158] Star,H[158] Total,H[15] Gas,H[15] Star,H[15] Total,H[160] Gas,H[160] Star,H[160] Total,H[16] Gas,H[16] Star,H[16] Total,H[170] Gas,H[170] Star,H[170] Total,H[17] Gas,H[17] Star,H[17] Total,H[183] Gas,H[183] Star,H[183] Total,H[19] Gas,H[19] Star,H[19] Total,H[1] Gas,H[1] Star,H[1] Total,H[20] Gas,H[20] Star,H[20] Total,H[21] Gas,H[21] Star,H[21] Total,H[22] Gas,H[22] Star,H[22] Total,H[23] Gas,H[23] Star,H[23] Total,H[24] Gas,H[24] Star,H[24] Total,H[25] Gas,H[25] Star,H[25] Total,H[26] Gas,H[26] Star,H[26] Total,H[27] Gas,H[27] Star,H[27] Total,H[2] Gas,H[2] Star,H[2] Total,H[302] Gas,H[302] Star,H[302] Total,H[310] Gas,H[310] Star,H[310] Total,H[312] Gas,H[312] Star,H[312] Total,H[319] Gas,H[319] Star,H[319] Total,H[31] Gas,H[31] Star,H[31] Total,H[324] Gas,H[324] Star,H[324] Total,H[325] Gas,H[325] Star,H[325] Total,H[32] Gas,H[32] Star,H[32] Total,H[337] Gas,H[337] Star,H[337] Total,H[33] Gas,H[33] Star,H[33] Total,H[35] Gas,H[35] Star,H[35] Total,H[36] Gas,H[36] Star,H[36] Total,H[3] Gas,H[3] Star,H[3] Total,H[40] Gas,H[40] Star,H[40] Total,H[41] Gas,H[41] Star,H[41] Total,H[42] Gas,H[42] Star,H[42] Total,H[44] Gas,H[44] Star,H[44] Total,H[45] Gas,H[45] Star,H[45] Total,H[469] Gas,H[469] Star,H[469] Total,H[47] Gas,H[47] Star,H[47] Total,H[484] Gas,H[484] Star,H[484] Total,H[488] Gas,H[488] Star,H[488] Total,H[48] Gas,H[48] Star,H[48] Total,H[499] Gas,H[499] Star,H[499] Total,H[4] Gas,H[4] Star,H[4] Total,H[503] Gas,H[503] Star,H[503] Total,H[504] Gas,H[504] Star,H[504] Total,H[516] Gas,H[516] Star,H[516] Total,H[523] Gas,H[523] Star,H[523] Total,H[53] Gas,H[53] Star,H[53] Total,H[57] Gas,H[57] Star,H[57] Total,H[5] Gas,H[5] Star,H[5] Total,H[631] Gas,H[631] Star,H[631] Total,H[63] Gas,H[63] Star,H[63] Total,H[649] Gas,H[649] Star,H[649] Total,H[65] Gas,H[65] Star,H[65] Total,H[67] Gas,H[67] Star,H[67] Total,H[6] Gas,H[6] Star,H[6] Total,H[70] Gas,H[70] Star,H[70] Total,H[7] Gas,H[7] Star,H[7] Total,H[8] Gas,H[8] Star,H[8] Total,H[90] Gas,H[90] Star,H[90] Total,H[93] Gas,H[93] Star,H[93] Total,H[95] Gas,H[95] Star,H[95] Total,H[97] Gas,H[97] Star,H[97] Total,H[9] Gas,H[9] Star,H[9] Total
snap,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.139588e+09,1.353323e+08,1.057458e+10,7.503453e+06,1.043767e+08,9.287775e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.674600e+11,1.882703e+11,2.109123e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.983606e+09,1.502495e+09,4.858006e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.844133e+08,4.313225e+08,3.328009e+10,NaN,NaN,NaN,NaN,NaN,NaN,0.0,119724.141995,8.022597e+07,NaN,NaN,NaN,NaN,NaN,NaN,1.012777e+09,3.315968e+08,3.058476e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.193793e+0

In [204]:
iord3510=pd.DataFrame(columns=['gas', 'star', 'total'])
iord3510

,gas,star,total


In [210]:
ind=0
for i in iords.index:
    iord3510.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863510'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863510'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863510'])]]
    ind=ind+1

In [213]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [214]:
iord3727=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3727.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863727'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863727'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863727'])]]
    ind=ind+1

KeyError: 'H[8.0] Gas'

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [ ]:
iord3565=pd.DataFrame(columns=['gas', 'star', 'total'])
ind=0
for i in iords.index:
    iord3565.loc[int(iords.index[ind]),['gas','star','total']]=[masses.at[int(iords.index[ind]),'H[%s] Gas'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Star'%(iords.at[i,'101863565'])],masses.at[int(iords.index[ind]),'H[%s] Total'%(iords.at[i,'101863565'])]]
    ind=ind+1

In [67]:
pynbody.plot.generic.hist2d(snap,masses)

NameError: name 'snap' is not defined

In [81]:
masses['H[1] Gas'][4096]

167460004583.0

In [212]:
iords

,101863510,101863565,101863705,101863726,101863727,101863732,101863734,101863739,101863741,101863769,101863883,101864796,101865286,101865398,101866127,101867516
snap,,,,,,,,,,,,,,,,
0139,5,3,3.0,10,8.0,33,183,14,2.0,53.0,19,9,1,6,26,NaN
0225,4,8,8.0,11,5.0,23,302,27,10.0,2.0,26,9,6,35,45,3.0
0275,5,10,10.0,27,7.0,21,337,19,15.0,1.0,41,12,8,40,57,3.0
0640,1,13,13.0,35,22.0,8,141,26,1.0,2.0,44,16,10,36,1,9.0
0974,1,6,6.0,1,22.0,11,95,10,1.0,1.0,16,15,2,32,1,7.0
1024,1,4,NaN,42,23.0,11,97,10,1.0,1.0,17,15,2,31,1,7.0
1269,1,4,NaN,67,4.0,1,90,7,1.0,1.0,14,12,2,24,1,6.0
1280,1,4,NaN,67,4.0,1,93,7,1.0,1.0,14,12,2,25,1,6.0
1408,1,3,NaN,1,3.0,24,109,6,1.0,1.0,11,12,2,23,1,5.0
